In [3]:
import pandas as pd
from df_gen import *

%load_ext autoreload
%autoreload 2

In [ ]:
# concatenate all .xyz files into the dataset file
! cat ../data/xyz/*-atoms.xyz > ../data/xyz/dataset-npt_anneal_293K_-1.xyz

In [2]:
# Convert this file to a dataframe. This requires ase==3.22.1 while the rest of the analysis requires ase==3.23.0
#df=convert_xyz_pickle('../data/xyz/dataset-npt_anneal_293K_-1.xyz')

In [23]:
df=pd.read_pickle('../data/xyz/dataset-npt_anneal_293K_-1.xyz.pkl.gzip',compression="gzip")
df.columns

Index(['energy', 'forces', 'ase_atoms'], dtype='object')

In [24]:
df=build_df_unary(df)
df.rename(columns={'energy':'mtp_energy'}, inplace=True)

In [42]:
# obtain config type and assign unique color to each category
df["Category_2"] = df['ase_atoms'].apply(config_sort_v2)
color_map = {cat:cycle_colors(ii) for ii,cat in enumerate(df["Category_2"].unique())}
df["Category_color_2"] = df["Category_2"].map(color_map)
df['size']=df['nb_atoms'].apply(marker_size)

In [37]:
# get energy relative to c-Si
df['dE_mtp']=df['mtp_energy']+(163161.251478124/1000) # energy of a relaxed configuration of 1,000 c-Si atoms
df['mtp_at_E_NN']=df['ase_atoms'].apply(NN_E_mtp)

In [40]:
# Descriptors used in our analysis
df["soap_sim_cSi"] = df['ase_atoms'].apply(structure_soap_cSi)
df["atomistic_soap_sim_cSi"] = df['ase_atoms'].apply(atomistic_soap_cSi)
df["CNA"]=df['ase_atoms'].apply(CNA)

In [46]:
# Crystalline/amorph
df['ptm_cat']=df.apply(ptm_cat, axis=1)


In [51]:
df.columns

Index(['mtp_energy', 'forces', 'ase_atoms', 'vol_per_atom', 'F_max',
       'nb_atoms', 'label', 'Category_2', 'Category_color_2', 'size', 'dE_mtp',
       'soap_sim_cSi', 'mtp_at_E_NN', 'atomistic_soap_sim_cSi', 'CNA',
       'ptm_cat'],
      dtype='object')

In [53]:
# Export dataframe
cols=['ase_atoms', 'nb_atoms', 'size', 'vol_per_atom', 'label', 'Category_2', 'Category_color_2', 'mtp_energy', 'dE_mtp', 'mtp_at_E_NN', 'forces',  'F_max', 'soap_sim_cSi', 'atomistic_soap_sim_cSi', 'CNA', 'ptm_cat']
df_data=df[cols]
df_data.to_pickle("../data/xyz/df_npt_anneal_293K_10ps_-1_generated.pckl.gzip", compression="gzip", protocol=4)